In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
NBC_df = pd.read_excel(r'C:\Users\bhavrang\Desktop\bhavya\Analytics\NBCdata.xlsx')

In [138]:
#to get the count of gold_subscription_ids per user.
freqdf=NBC_df.groupby('USER_DM_ID')['GOLD_SUBSCRIPTION_ID'].count().reset_index(name='GOLD_ID_COUNT')

In [139]:
freqdf.GOLD_ID_COUNT.quantile([0.25,0.5,0.75])

0.25    1.0
0.50    1.0
0.75    1.0
Name: GOLD_ID_COUNT, dtype: float64

In [140]:
#to apply score for a user based on counts of gold_subscription_ids
freqdf['FREQ_SCORE']=freqdf['GOLD_ID_COUNT'].apply(lambda x: 2 if x>1 else 1)

In [15]:
#Monetary
#summing up all the active subscription price at user level.
price=NBC_df[NBC_df['DATA_SOURCE_STATUS']=='ACTIVE'].groupby('USER_DM_ID')['SUBCRIPTION_PRICE'].sum().reset_index(name='TOTAL_PRICE')

In [16]:
#summing up all refund amount at user level.
ref_price=NBC_df[NBC_df['DATA_SOURCE_STATUS']!='ACTIVE'].groupby('USER_DM_ID')['TOTAL_REFUNDED_AMT'].sum().reset_index(name='TOTAL_REFUND')

In [17]:
price=pd.merge(price,ref_price,how='left',on='USER_DM_ID')
price=price.fillna(0)
price['REVENUE']=price['TOTAL_PRICE']-price['TOTAL_REFUND']

In [131]:
#scoring user based on REVENUE generated 
def monetory_score(x):
    if(x>100):
        return 5
    elif(50<=x<100):
        return 4
    elif(20<=x<50):
        return 3
    elif(10<=x<20):
        return 2
    elif(1<=x<10):
        return 1
    else :
        return 0

In [132]:
price['MONETARY_SCORE']=price['REVENUE'].apply(lambda x:monetory_score(x))

In [103]:
#scoring based on recency
recency_df=NBC_df[['USER_DM_ID','SUBSCRIPTION_START_DT']].copy()

In [104]:
recency_df['SUBSCRIPTION_START_DT']=recency_df['SUBSCRIPTION_START_DT'].apply(lambda x:np.datetime64(x))
recency_df['DATE_DIFF']= np.datetime64('2018-12-31')-recency_df['SUBSCRIPTION_START_DT']
recency_df['RECENCY']=recency_df['DATE_DIFF'].apply(lambda x: x.days)

In [105]:
recency_df=recency_df.groupby('USER_DM_ID').agg({'RECENCY':'min'})

In [107]:
recency_df.loc[100053]

RECENCY    226
Name: 100053, dtype: int64

In [114]:
rec=NBC_df.groupby('USER_DM_ID')['GOLD_SUBSCRIPTION_ID'].count().reset_index(name='count')

In [115]:
rec=pd.merge(rec,recency_df,on='USER_DM_ID',how='left')

In [111]:
def recency_score(x):
    if(0<=x<=92):
        return 4
    elif(92<x<=184):
        return 3
    elif(184<x<=275):
        return 2
    elif(x>275):
        return 1

In [116]:
rec['RECENCY_SCORE']=rec['RECENCY'].apply(lambda x:score(x))

In [120]:
rec.drop(['count'],inplace=True,axis=1)

In [145]:
final_df=pd.merge(freqdf[['USER_DM_ID','FREQ_SCORE']],price[['USER_DM_ID','MONETARY_SCORE']],on='USER_DM_ID').merge(rec[['USER_DM_ID','RECENCY_SCORE']],on='USER_DM_ID')

In [ ]:
r * 20 + f * 50 + m * 30

In [153]:
final_df.head()

,USER_DM_ID,FREQ_SCORE,MONETARY_SCORE,RECENCY_SCORE,RFM_SCORE
0,1,1,3,3,200
1,3,1,3,2,180
2,4,1,4,2,210
3,5,1,3,3,200
4,6,1,3,2,180


In [152]:
final_df['RFM_SCORE']=final_df['FREQ_SCORE']*50 + final_df['MONETARY_SCORE']*30 + final_df['RECENCY_SCORE']*20

In [150]:
final_df.to_excel(excel_writer=r'C:\Users\bhavrang\Desktop\bhavya\Analytics\NBCdata_RFMscore.xlsx')